# Lab | Making predictions with logistic regression

In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented. We are asked to create a model to predict it. So we use the information we have from May 2005 to create the model.

### Instructions

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X). 
2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our **TARGET** (y) variable.
3. Read the data into a Pandas dataframe.  At this point you should have 1000 rows.  Number of columns depends on the number of features you chose.
4. Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict 'rented_in_may' from the cleaned data.
6. Evaluate the results.

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
# query = '''SELECT
#     f.rental_duration, f.rental_rate, f.length, f.rating, f.replacement_cost,
#     MAX(IF(MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005, 1, 0)) AS rented_in_may
# FROM
#     film AS f
# LEFT JOIN
#     inventory AS i ON f.film_id = i.film_id
# LEFT JOIN
#     rental AS r ON i.inventory_id = r.inventory_id
# GROUP BY
#     f.title;'''

query = '''SELECT
    f.rental_duration, f.rental_rate, f.length, f.rating, p.amount,
    MAX(IF(MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005, 1, 0)) AS rented_in_may
FROM
    film AS f
LEFT JOIN
    inventory AS i ON f.film_id = i.film_id
LEFT JOIN
    rental AS r ON i.inventory_id = r.inventory_id
LEFT JOIN
    payment AS p ON r.rental_id = p.rental_id
GROUP BY
    f.title;'''

# query = '''SELECT
#     f.rental_duration, f.rental_rate, f.length, f.rating, f.replacement_cost, a.last_name,
#     MAX(IF(MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005, 1, 0)) AS rented_in_may
# FROM
#     film AS f
# LEFT JOIN
#     inventory AS i ON f.film_id = i.film_id
# LEFT JOIN
#     rental AS r ON i.inventory_id = r.inventory_id
# LEFT JOIN
#     film_actor AS fc ON f.film_id = fc.film_id
# LEFT JOIN
#     actor AS a ON a.actor_id = fc.actor_id
# GROUP BY
#     f.title;'''

data = pd.read_sql_query(query, engine)
data.head(10)

,rental_duration,rental_rate,length,rating,amount,rented_in_may
0,6,0.99,86,PG,0.99,1
1,3,4.99,48,G,4.99,0
2,7,2.99,50,NC-17,2.99,1
3,5,2.99,117,G,2.99,1
4,6,2.99,130,G,3.99,1
5,3,2.99,169,PG,5.99,1
6,6,4.99,62,PG-13,4.99,0
7,6,4.99,54,R,5.99,1
8,3,2.99,114,PG-13,8.99,0
9,6,4.99,63,NC-17,4.99,0


In [3]:
data['amount'] = data['amount'].fillna(3.99)

In [4]:
data.isna().sum()

rental_duration    0
rental_rate        0
length             0
rating             0
amount             0
rented_in_may      0
dtype: int64

In [5]:
data.dtypes

rental_duration      int64
rental_rate        float64
length               int64
rating              object
amount             float64
rented_in_may        int64
dtype: object

In [ ]:
correlations_matrix = data.select_dtypes(include = np.number).corr()
sns.heatmap(correlations_matrix, annot=True)
plt.show()

In [ ]:
y = data['rented_in_may']
X = data.drop('rented_in_may', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(include = np.number)

# Scaling data
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized, columns=X_train_num.columns)
X_train_norm

In [ ]:
correlations_matrix = X_train_num.corr()
sns.heatmap(correlations_matrix, annot=True)
plt.show()

In [ ]:
X_train_categorical = X_train.select_dtypes(include = object)
X_train_cat = pd.get_dummies(X_train_categorical, 
                             columns=['rating'],
                             drop_first=True)
X_train_cat

In [ ]:
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)
X_train_transformed.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [ ]:
X_test_num = X_test.select_dtypes(include = np.number)

# Scaling data
# we use the transformer that was trained on the training data
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)
X_test_norm

In [ ]:
X_test_categorical = X_test.select_dtypes(include = object)
X_test_cat = pd.get_dummies(X_test_categorical, 
                            columns=['rating'],
                            drop_first=True)
X_test_cat

In [ ]:
list(X_train_cat.columns)==list(X_test_cat.columns)

In [ ]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)
X_test_transformed.shape

In [ ]:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)